# Day 15: Chiton

You've almost reached the exit of the cave, but the walls are getting closer together. Your submarine can barely still fit, though; the main problem is that the walls of the cave are covered in chitons, and it would be best not to bump any of them.

The cavern is large, but has a very low ceiling, restricting your motion to two dimensions. The shape of the cavern resembles a square; a quick scan of chiton density produces a map of risk level throughout the cave (your puzzle input).

## Input

In [ ]:
using System.IO;
// var input = File.ReadAllLines(@"day-15.sample");
var input = File.ReadAllLines(@"day-15.input");

var risks = input.Select(l => l.ToCharArray()
        .Select(c => (int)(c - '0')).ToArray()
    ).ToArray();

(risks[0].Length, risks.Length)

Item1,Item2
100,100


## Part 1

You start in the top left position, your destination is the bottom right position, and you cannot move diagonally. The number at each position is its risk level; to determine the total risk of an entire path, add up the risk levels of each position you enter (that is, don't count the risk level of your starting position unless you enter it; leaving it adds no risk to your total).

Your goal is to find a path with the lowest total risk.

What is the lowest total risk of any path from the top left to the bottom right?

In [ ]:
enum Dir { none, left, up, right, down };

record struct PathRisk(int Risk, int MinRisk, Dir From) {
    public void Add(PathRisk p, Dir from) {
        var newRisk = p.MinRisk + Risk;
        if (newRisk < MinRisk) {
            From = from;
            MinRisk = newRisk;
        }
    }

    public override string ToString()
        => From switch {
                Dir.left => $"<{MinRisk:d4},{Risk:d}",
                Dir.up => $"^{MinRisk:d4},{Risk:d}",
                Dir.right => $">{MinRisk:d4},{Risk:d}",
                Dir.down => $"v{MinRisk:d4},{Risk:d}",
                _ => "    "
            };
}

var pathRisks = risks.Select(l => l.Select(r => new PathRisk(r, 9999, Dir.none)).ToArray()).ToArray();
pathRisks[0][0].MinRisk = 0;

int CalculateMinRisk(int show) {
    var size = pathRisks.Length;
    var last = size - 1;

    for (var y = 0; y < size; y++)
        for (var x = 0; x < size; x++) {
            if (x > 0) pathRisks[y][x].Add(pathRisks[y][x-1], Dir.left);
            if (y > 0) pathRisks[y][x].Add(pathRisks[y-1][x], Dir.up);
            if (x < last) pathRisks[y][x].Add(pathRisks[y][x+1], Dir.right);
            if (y < last) pathRisks[y][x].Add(pathRisks[y+1][x], Dir.down);
        }

    foreach (var r in pathRisks.TakeLast(show)) {
        var row = string.Join(" ", r.TakeLast(show).Select(p => p.ToString()));
        Console.WriteLine(row);
    }

    return pathRisks[last][last].MinRisk;
}

CalculateMinRisk(4)


^0677,8 ^0681,6 ^0677,8 ^0682,8
^0680,3 <0684,4 ^0683,6 ^0691,9
^0683,3 <0690,7 ^0686,3 <0693,7
^0684,1 <0685,1 <0693,8 <0702,9


702

## Part 2
Now that you know how to find low-risk paths in the cave, you can try to find your way out.

The entire cave is actually five times larger in both dimensions than you thought; the area you originally scanned is just one tile in a 5x5 tile area that forms the full map. Your original map tile repeats to the right and downward; each time the tile repeats to the right or downward, all of its risk levels are 1 higher than the tile immediately up or left of it. However, risk levels above 9 wrap back around to 1.

Using the full map, what is the lowest total risk of any path from the top left to the bottom right?

In [ ]:
static int IncRisk(int risk, int extra)
    => (risk - 1 + extra) % 9 + 1;

pathRisks = Enumerable.Range(0, 5).SelectMany(y => risks.Select(l => 
    Enumerable.Range(0, 5).SelectMany(x => l.Select(r => new PathRisk(IncRisk(r, x + y), 9999, Dir.none))).ToArray()
)).ToArray();
pathRisks[0][0].MinRisk = 0;

var (oldRisk, minRisk) = (9999, CalculateMinRisk(2));

while (minRisk < oldRisk) {
    (oldRisk, minRisk) = (minRisk, CalculateMinRisk(2));
}

CalculateMinRisk(2)

<2953,2 <2959,6
^2960,7 ^2967,8
<2951,2 <2957,6
^2958,7 ^2965,8
<2949,2 <2955,6
^2956,7 ^2963,8
<2948,2 <2954,6
^2955,7 ^2962,8
<2946,2 <2952,6
^2953,7 ^2960,8
<2945,2 <2951,6
^2952,7 ^2959,8
<2944,2 <2950,6
^2951,7 ^2958,8
<2943,2 <2949,6
^2950,7 ^2957,8
<2942,2 <2948,6
^2949,7 ^2956,8
<2941,2 <2947,6
^2948,7 ^2955,8
<2941,2 <2947,6
^2948,7 ^2955,8
<2941,2 <2947,6
^2948,7 ^2955,8


2955